## Instal·lant dependències

In [3]:
import os
from fastai import *
from fastai.text import *
from gensim.corpora import WikiCorpus

import numpy as np
import pandas as pd

from tqdm import tqdm
from pathlib import Path
from functools import partial

In [ ]:
!pip install sacremoses

In [ ]:
!nvidia-smi

## Descarreguem el Wikipedia dump en català

In [ ]:
!rm -rf data/wiki_extr
!rm -rf wikiextractor/ WikiExtractor.py

In [ ]:
!sh get_wikimedia.sh

In [ ]:
name = 'CatalanULMFit'
model_dir = 'models'
model_dir = Path(model_dir)
model_dir.mkdir(exist_ok=True)

Creem el corpus

In [ ]:
!ls data/wiki_extr/ca

In [ ]:
!python create_wikiText.py -i data/wiki_extr/ca -o data/wiki -l ca

In [4]:
trn_path = Path('data/wiki/ca-100/ca.wiki.train.tokens')
val_path = Path('data/wiki/ca-100/ca.wiki.valid.tokens')

In [5]:
with open(trn_path, encoding='utf8') as f:
     train = f.readlines()

In [ ]:
df_train = pd.DataFrame(columns=['content', 'is_valid'])
for line in tqdm(train):
    df_train = df_train.append({'content':line, 'is_valid': 0},ignore_index=True)
print('df_train done')

  9%|▉         | 301743/3382970 [1:09:25<19:40:54, 43.49it/s]

In [ ]:
del(train)

In [ ]:
with open(val_path, encoding='utf8') as f:
      val = f.readlines()

In [ ]:
df_valid = pd.DataFrame(columns=['content', 'is_valid'])
for line in tqdm(val):
    df_valid = df_valid.append({'content':line, 'is_valid': 0},ignore_index=True)
print('df_valid done')

In [ ]:
del(val)

In [ ]:
df_regroup = pd.concat([df_train, df_valid])
df_regroup.to_csv('data/wiki/full_train.csv', header=None, index=None)

In [ ]:
 !python postprocess_wikiText.py -i data/wiki -l ca

In [ ]:
from tqdm import tqdm

In [ ]:
tokenizer = Tokenizer(lang='ca', n_cpus=6)
data_lm_full = (TextList.from_csv('data/wiki', csv_name='full_train.csv', cols=0, processor=[TokenizeProcessor(tokenizer=tokenizer), NumericalizeProcessor(max_vocab=60000)])
           #Inputs: all the text files in path
            .split_from_df(col=1)
           #We may have other temp folders that contain text files so we only keep what's in train and test
            .label_for_lm()           
           #We want to do a language model so we label accordingly
            .databunch(bs=32))


In [ ]:
!du -H data/wiki/full_train.csv

In [ ]:
itos = data_lm_full.train_ds.vocab.itos
print('Size of vocabulary:', len(itos))
print('First 10 words in vocab:', ', '.join([itos[i] for i in range(10)]))

In [ ]:
torch.cuda.is_available()

# Entrenar el Language Model

Hem usat com a dataset tot el text que s'ha trobat en el repositori https://github.com/Softcatala/ca-text-corpus. També hem provat d'entrenar amb un dataset creat a partir del script **./get_wikimedia.sh** però ens tornava un dataset massa brut. 

In [ ]:
learn = language_model_learner(data_lm_full,  arch=AWD_LSTM, drop_mult=0, callback_fns=ShowGraph)

In [ ]:
# save vocabulary
print('Saving vocabulary...')
with open(os.path.join('data', 'itos.pkl'), 'wb') as f:
    pickle.dump(itos, f)

In [ ]:
data_lm_full.show_batch()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(skip_start=0)

In [ ]:
learn.fit_one_cycle(10, 1e-2, moms=(0.8, 0.7))

In [ ]:
 learn.fit_one_cycle(5, 5e-3, moms=(0.8, 0.7))

In [ ]:
 learn.fit_one_cycle(1, 5e-3, moms=(0.8, 0.7))

In [ ]:
learn.save('model_60k-vocab')

In [ ]:
np.exp(3.25)